In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
from scipy.ndimage import binary_closing, binary_opening, measurements

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

classes = ['large_bowel', 'small_bowel', 'stomach']
data_dir = "../input/uw-madison-gi-tract-image-segmentation/"
test_dir = os.path.join(data_dir, "test")
sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)

if len(test_images) == 0:
    test_dir = os.path.join(data_dir, "train")
    sub = pd.read_csv(os.path.join(data_dir, "train.csv"))[["id", "class"]].iloc[:100 * 3]
    sub["predicted"] = ""
    test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)
    
id2img = {_.rsplit("/", 4)[2] + "_" + "_".join(_.rsplit("/", 4)[4].split("_")[:2]): _ for _ in test_images}
sub["file_name"] = sub.id.map(id2img)
sub["days"] = sub.id.apply(lambda x: "_".join(x.split("_")[:2]))
fname2index = {f + c: i for f, c, i in zip(sub.file_name, sub["class"], sub.index)}
sub


/home/zhangzr/anaconda3/envs/mmseg-kaggle/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,class,predicted,file_name,days
0,case123_day20_slice_0001,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
1,case123_day20_slice_0001,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
2,case123_day20_slice_0001,stomach,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
3,case123_day20_slice_0002,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
4,case123_day20_slice_0002,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
...,...,...,...,...,...
295,case123_day20_slice_0099,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
296,case123_day20_slice_0099,stomach,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
297,case123_day20_slice_0100,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
298,case123_day20_slice_0100,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20


In [3]:
def convert_pred(img):
    '''
    (H, W) -> (H,W,Class)
    '''
    H = img.shape[0]
    W = img.shape[1]
    new_pred = np.zeros((H, W, 3))
    for i in range(H):
        for j in range(W):
            if img[i,j] == 1:
                new_pred[i,j,1] = 1
            if img[i,j] == 2:
                new_pred[i,j,2] = 1
    return new_pred

In [5]:
# kaggle version
for day, group in tqdm(sub.groupby("days")):
    imgs = []
    for file_name in group.file_name.unique():
        res = cv2.imread(file_name, cv2.IMREAD_UNCHANGED)
        res = convert_pred(res)
        for j in range(3):
            rle = rle_encode(res[...,j])
            index = fname2index[file_name + classes[j]]
            sub.loc[index, "predicted"] = rle

100%|██████████| 1/1 [00:55<00:00, 55.59s/it]


In [6]:
# local version
pred_result_root_path = '../test_results/upernet_convnext_base_fp16_512x512_160k_kaggle'
pred_img_name_list = os.listdir(pred_result_root_path)
for pred_img_name in tqdm(pred_img_name_list):
    pred_img = cv2.imread(os.path.join(pred_result_root_path, pred_img_name), cv2.IMREAD_UNCHANGED)
    res = convert_pred(pred_img)
    for j in range(3):
        rle = rle_encode(res[...,j])
        #index = fname2index[pred_img + classes[j]]
        #sub.loc[index, "predicted"] = rle
       

  1%|          | 39/3776 [00:51<18:20,  3.40it/s]  

: 